## 1. Find duplicate record

In [0]:
%sql
select col1,col2,count(*)
from emp_tbl
group by col1,col2
having count(*)>1

In [0]:
from pyspark.sql.functions import count

emp_tbl.groupBy("col1", "col2") \
       .agg(count("*").alias("cnt")) \
       .filter("cnt > 1") \
       .show()


## 2.Second highest salary

In [0]:
%sql

select max(salary) as max_sal
from emp_tbl
where salary<(
select max(salary)
from emp_tbl

)

In [0]:
from pyspark.sql.functions import max, col

emp_tbl.filter(col("salary") < emp_tbl.agg(max("salary")).collect()[0][0]) \
       .agg(max("salary").alias("max_sal")) \
       .show()


## 3.Find emp without department

In [0]:
%sql
select emp.*
from emp e
left join dept d
on e.dept_id=d.dept_id
where d.dept_id is null

In [0]:
emp.alias("e").join(dept.alias("d"), on="dept_id", how="left") \
   .filter("d.dept_id IS NULL") \
   .select("e.*") \
   .show()


## 4.Calculate total revenue per product

In [0]:
%sql
select product_id,sum(quantity*price) as total_rev
from sales_tbl
group by product_id

In [0]:
from pyspark.sql.functions import sum

sales_tbl.groupBy("product_id") \
         .agg(sum("sales").alias("total_rev")) \
         .show()


## 5.Top 3 highest paid emp

In [0]:
%sql
select *
from emp_tbl
order by salary desc
limit 3

In [0]:
emp_tbl.orderBy("salary", ascending=False) \
       .limit(3) \
       .show()


## 6.Show count of orders per customers

In [0]:
%sql
select customer_id,count(*)
from order
group by customer_id

In [0]:
from pyspark.sql.functions import count

order.groupBy("customer_id") \
     .agg(count("*").alias("cnt")) \
     .show()


## 7.All emp who joined in 2023

In [0]:
%sql
select *
from emp_tbl
where YEAR(hire_date)=2023

In [0]:
from pyspark.sql.functions import year

emp_tbl.filter(year("hire_date") == 2023) \
       .show()


## 8.Calculate average order value per customer

In [0]:
%sql
select customer_id
,avg(total_amount) as avg_order_value
from orders
group by customer_id

In [0]:
from pyspark.sql.functions import avg

orders.groupBy("customer_id") \
      .agg(avg("total_amount").alias("avg_order_value")) \
      .show()


## 9.Get latest order places by each customer

In [0]:
%sql
select customer_id,max(order_date) as latest_order_date
from orders
group by customer_id

In [0]:
from pyspark.sql.functions import max

orders.groupBy("customer_id") \
      .agg(max("order_date").alias("latest_order_date")) \
      .show()


## 10.Find product that were never sold

In [0]:
%sql
select p.product_id,p.product_name
from product p
left join sales s
on p.product_id=s.product_id
where s.product_id is null

In [0]:
product.alias("p").join(sales.alias("s"), on="product_id", how="left") \
       .filter("s.product_id IS NULL") \
       .select("p.product_id", "p.product_name") \
       .show()


## 11.Find product that were never sold

In [0]:
%sql
SELECT p.product_id
FROM Products p
LEFT JOIN Sales s
ON p.product_id = s.product_id
WHERE s.product_id IS NULL;


In [0]:
products.alias("p").join(sales.alias("s"), on="product_id", how="left") \
        .filter("s.product_id IS NULL") \
        .select("p.product_id") \
        .show()


## 12.Identify the most selling product

In [0]:
%sql
select top 1 product_id, sum(quantity) as total_quantity
from sales
group by product_id
order by total_quantity desc

In [0]:
sales.groupBy("product_id") \
     .agg(sum("quantity").alias("total_quantity")) \
     .orderBy(col("total_quantity").desc()) \
     .limit(1) \
     .display()

## 13.Get the total revenue and number of orders per region

In [0]:
%sql
select region, sum(total_amount) as total_revenue,count(*) as order_count
from orders
group by region

In [0]:
orders.groupBy("region") \
      .agg(sum("total_amount").alias("total_revenue"),
          count("*").alias("order_count")) \
      .display()

## 14.Count how many customers placed more than 5 orders

In [0]:
%sql
select count(*) as customer_count
from(
  select customer_id
from orders
group by customer_id
having count(*)> 5
) as subquery

In [0]:
orders.groupBy("customer_id") \
      .count() \
      .filter("count > 5") \
      .count()

## 15. Retrieve customers orders above the average order value

In [0]:
%sql
select *
from orders
where total_amount>(
  select avg(total_amount)
  from orders
)

In [0]:
avg_amount = orders.agg(avg("total_amount")).collect()[0][0]

orders.filter(col("total_amount") > avg_amount) \
      .display()

## 16.Find all emp hired on weekends

In [0]:
%sql
select * from emp
where DATENAME(WEEKDAY, hire_date) in ('Saturday', 'Sunday')

In [0]:
emp.filter(date_format("hire_date", "E").isin("Sat", "Sun")) \
   .display()

## 18. Get monthly sale revenue and order date

In [0]:
%sql
select FORMAT(date,'yyyy-MM') as month,
sum(amount) as total_revenue,
count(order_id) as order_count
from orders
group by FORMAT(date,'yyyy-MM')

In [0]:
orders.groupBy(date_format("date", "yyyy-MM").alias("month")) \
        .agg(sum("amount").alias("total_revenue"),
          count("order_id").alias("order_count")) \
        .display()

## 19. Rank employees by salary by within each department

In [0]:
%sql
select emp_id,emp_name,depart_id,salary,dense_rank()over(partition by depart_id order by salary desc) as rank
from emp_tbl

In [0]:
emp_tbl.withColumn("rank",dense_rank().over(Window.partitionBy("depart_id").orderBy(col("salary").desc()))) \
      .select("emp_id", "emp_name", "depart_id", "salary", "rank") \
      .display()

##20.Find customer who placed order in every month

In [0]:
%sql
select customer_id
from orders
where year(order_date)=2023
group by customer_id
having count(distinct format(order_date,'yyyy-MM'))=12

In [0]:
orders.filter(year("order_date") == 2023) \
      .groupBy("customer_id") \
      .agg(countDistinct(date_format("order_date", "yyyy-MM")).alias("months_count")) \
      .filter(col("months_count") == 12) \
      .select("customer_id") \
      .display()

## 21. Find moving average of sales over the last 3 days.

In [0]:
%sql
select order_date,avg(total_amount) over (order by order_date rows between 2 preceding and current row) as moving_avg
from orders

In [0]:
orders.withColumn("moving_avg", avg("total_amount").over(Window.orderBy("order_date").rowsBetween(-2,0))) \
      .display()

## 22. Identify the first and last order date for each customer.

In [0]:
%sql
select customer_id
min(order_date) as first_order,
max(order_date) as last_order
from orders
group by customer_id

In [0]:
orders.groupBy("customer_id") \
      .agg(
          min("order_date").alias("first_order"),
          max("order_date").alias("last_order")) \
      .display()

## 24. Retrieve customers who made consecutive purchases (2 Days)

In [0]:
%sql
with cte as(
  select id,order_date,lag(order_date) over(partition by id order by order_date) as pre_order_date
  from orders

)

select id,order_date,pre_order_date
from cte
where datediff(day,pre_order_date,order_date)=1

In [0]:
cte = orders.withColumn(
          "pre_order_date",
          lag("order_date").over(Window.partitionBy("id").orderBy("order_date"))
      )

cte.filter(datediff(col("order_date"), col("pre_order_date")) == 1) \
   .select("id", "order_date", "pre_order_date") \
   .display()

## 25. Find churned customers (no orders in the last 6 months).

In [0]:
%sql
select customer_id
from orders
group by customer_id
having max(order_date)< dateadd(month,-6,getdate())

In [0]:
cutoff = add_months(current_date(), -6)

orders.groupBy("customer_id") \
      .agg(max_("order_date").alias("last_order_date")) \
      .filter(col("last_order_date") < cutoff) \
      .select("customer_id") \
      .display()

## 26. Calculate cumulative revenue by day.

In [0]:
%sql
select order_date, sum(amount) over(order by order_date) as total_rev
from orders

In [0]:
orders.withColumn("total_rev",sum("amount").over(Window.orderBy("order_date")))\
      .display()

## 27. Identify top-performing departments by average salary.

In [0]:
%sql
select dep_id,avg(salary) as avg_salary
from emp
group by dep_id
order by avg_salary


In [0]:
emp.groupBy("dep_id") \
   .agg(avg("salary").alias("avg_salary")) \
   .orderBy("avg_salary") \
   .display()

## 28. Find customers who ordered 
## more than the average number of orders per customer.


In [0]:
%sql
WITH customer_orders AS (
SELECT customer_id, COUNT(*) AS order_count
FROM Orders
GROUP BY customer_id)
SELECT * FROM customer_orders
WHERE order_count > (SELECT
AVG(order_count) FROM customer_orders);

In [0]:
customer_orders = orders_df.groupBy("customer_id") \
                           .agg(count("*").alias("order_count"))

avg_val = customer_orders.agg(avg("order_count")).collect()[0][0]

customer_orders.filter(col("order_count") > avg_val) \
               .display()

## 30. Find the percentage of employees in each department.

In [0]:
%sql
SELECT
department_id,
COUNT(*) AS emp_count,
COUNT(*) * 100.0 / (SELECT
COUNT(*) FROM Employee)
AS pct FROM Employee
GROUP BY department_id;

In [0]:
total_count = employee_df.count()

employee_df.groupBy("department_id") \
           .agg(count("*").alias("emp_count")) \
           .withColumn("pct", col("emp_count") * 100.0 / total_count) \
           .display()

## 31. Retrieve the maximum salary difference within each department.

In [0]:
%sql
SELECT
department_id,
MAX(salary) - MIN(salary) AS
salary_diff
FROM Employee
GROUP BY department_id;

In [0]:
emp_df.groupBy("dept_id").agg((max("salary"))-min("salary").alias("sal_diff"))\
  .display()

## 35. Calculate year-over-year growth in revenue.

In [0]:
%sql
SELECT FORMAT(order_date, 'yyyy') AS year,
SUM(total_amount) AS revenue,
SUM(total_amount) - LAG(SUM(total_amount))
OVER (ORDER BY FORMAT(order_date, 'yyyy'))
AS yoy_growth
FROM Orders
GROUP BY FORMAT(order_date, 'yyyy');

In [0]:
orders_df.withColumn("year", date_format("order_date", "yyyy")) \
         .groupBy("year") \
         .agg(sum("total_amount").alias("revenue")) \
         .withColumn("yoy_growth",
                     col("revenue") - lag("revenue").over(Window.orderBy("year"))) \
         .display()
